# Atualização do tccfinal.ipynb com correções e melhores práticas

## 0. Instalação de Bibliotecas (Se necessário)

Certifique-se de que o TensorFlow está instalado no seu ambiente. Se não estiver, execute a célula abaixo:

In [ ]:
%pip install tensorflow matplotlib pandas numpy scipy

## 1. Importar Bibliotecas Necessárias

In [2]:

import tensorflow as tf
from tensorflow.keras import datasets, layers, models, regularizers, callbacks
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
import os

## 2. Hiperparâmetros e Configuração

In [3]:
NUM_RUNS = 3      # Número de execuções para significância estatística
EPOCHS = 50       # Número original de épocas
PATIENCE = 5      # Paciência para Early Stopping
BATCH_SIZE = 32   # Tamanho do lote
L2_FACTOR = 0.001 # Fator de regularização L2
DROPOUT_RATE = 0.5 # Taxa de Dropout
LEARNING_RATE = 0.001 # Taxa de aprendizado

## 3. Carregamento e Pré-processamento do Dataset CIFAR-10

In [ ]:
(x_train_full, y_train_full), (x_test, y_test) = datasets.cifar10.load_data()

# Normalizar valores dos pixels para o intervalo [0, 1]
x_train_full, x_test = x_train_full / 255.0, x_test / 255.0

# Dividir dados de treinamento completos em treino e validação
val_split = 10000 # Usar 10000 amostras para validação
x_train, x_val = x_train_full[val_split:], x_train_full[:val_split]
y_train, y_val = y_train_full[val_split:], y_train_full[:val_split]

print(f"Amostras de Treinamento: {len(x_train)}")
print(f"Amostras de Validação: {len(x_val)}")
print(f"Amostras de Teste: {len(x_test)}")

# Nomes das classes para plotagem
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

## 4. Funções de Visualização

In [5]:
def plot_history(history, title):
    """Plota curvas de acurácia e perda para um único histórico de treinamento."""
    acc = history.history.get('accuracy')
    val_acc = history.history.get('val_accuracy')
    loss = history.history.get('loss')
    val_loss = history.history.get('val_loss')

    has_acc = acc is not None and val_acc is not None
    has_loss = loss is not None and val_loss is not None

    if not has_acc and not has_loss:
        print(f"Aviso: Histórico vazio ou incompleto para '{title}'. Não é possível plotar.")
        return

    epochs_with_data = 0
    if has_acc:
        epochs_with_data = len(acc)
    elif has_loss:
        epochs_with_data = len(loss)

    if epochs_with_data == 0:
        print(f"Aviso: Histórico sem épocas registradas para '{title}'. Não é possível plotar.")
        return

    epochs_range = range(1, epochs_with_data + 1)

    plt.figure(figsize=(12, 5))

    if has_acc:
        plt.subplot(1, 2, 1)
        plt.plot(epochs_range, acc, 'o-', label='Acurácia Treinamento', markersize=4)
        plt.plot(epochs_range, val_acc, '-', label='Acurácia Validação')
        plt.title(f'Acurácia - {title}')
        plt.xlabel('Épocas')
        plt.ylabel('Acurácia')
        plt.legend()
        plt.grid(True)
    else:
        plt.subplot(1, 2, 1)
        plt.title(f'Acurácia - {title}')
        plt.text(0.5, 0.5, 'Dados de Acurácia Indisponíveis', horizontalalignment='center', verticalalignment='center')
        plt.xlabel('Épocas')
        plt.ylabel('Acurácia')

    if has_loss:
        plt.subplot(1, 2, 2)
        plt.plot(epochs_range, loss, 'o-', label='Perda Treinamento', markersize=4)
        plt.plot(epochs_range, val_loss, '-', label='Perda Validação')
        plt.title(f'Perda - {title}')
        plt.xlabel('Épocas')
        plt.ylabel('Perda')
        plt.legend()
        plt.grid(True)
    else:
        plt.subplot(1, 2, 2)
        plt.title(f'Perda - {title}')
        plt.text(0.5, 0.5, 'Dados de Perda Indisponíveis', horizontalalignment='center', verticalalignment='center')
        plt.xlabel('Épocas')
        plt.ylabel('Perda')

    plt.suptitle(f'Desempenho do Modelo: {title}', fontsize=14)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

def plot_overfitting_gap(history, title):
    """Plota a diferença entre as métricas de treino e validação."""
    acc = history.history.get('accuracy')
    val_acc = history.history.get('val_accuracy')
    loss = history.history.get('loss')
    val_loss = history.history.get('val_loss')

    has_acc = acc is not None and val_acc is not None
    has_loss = loss is not None and val_loss is not None

    if not has_acc and not has_loss:
        return

    epochs_with_data = 0
    if has_acc:
        epochs_with_data = min(len(acc), len(val_acc))
    elif has_loss:
        epochs_with_data = min(len(loss), len(val_loss))

    if epochs_with_data == 0:
        return

    epochs_range = range(1, epochs_with_data + 1)

    plt.figure(figsize=(12, 5))

    if has_acc:
        acc_gap = [t - v for t, v in zip(acc[:epochs_with_data], val_acc[:epochs_with_data])]
        plt.subplot(1, 2, 1)
        plt.plot(epochs_range, acc_gap, '-', label='Gap Acurácia (Treino - Val)')
        plt.title(f'Gap de Acurácia - {title}')
        plt.xlabel('Épocas')
        plt.ylabel('Acurácia Treino - Acurácia Validação')
        plt.legend()
        plt.grid(True)
    else:
        plt.subplot(1, 2, 1)
        plt.title(f'Gap de Acurácia - {title}')
        plt.text(0.5, 0.5, 'Dados Indisponíveis', horizontalalignment='center', verticalalignment='center')
        plt.xlabel('Épocas')
        plt.ylabel('Acurácia Treino - Acurácia Validação')

    if has_loss:
        loss_gap = [v - t for t, v in zip(loss[:epochs_with_data], val_loss[:epochs_with_data])]
        plt.subplot(1, 2, 2)
        plt.plot(epochs_range, loss_gap, '-', label='Gap Perda (Val - Treino)')
        plt.title(f'Gap de Perda - {title}')
        plt.xlabel('Épocas')
        plt.ylabel('Perda Validação - Perda Treino')
        plt.legend()
        plt.grid(True)
    else:
        plt.subplot(1, 2, 2)
        plt.title(f'Gap de Perda - {title}')
        plt.text(0.5, 0.5, 'Dados Indisponíveis', horizontalalignment='center', verticalalignment='center')
        plt.xlabel('Épocas')
        plt.ylabel('Perda Validação - Perda Treino')

    plt.suptitle(f'Análise de Overfitting (Gap Treino-Validação): {title}', fontsize=14)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

def compare_histories(histories_dict, title="Comparação de Acurácia de Validação (Última Execução)"):
    plt.figure(figsize=(10, 6))
    plot_successful = False
    for label, history in histories_dict.items():
        if history is not None and 'val_accuracy' in history.history and history.history['val_accuracy']:
            epochs_range = range(1, len(history.history['val_accuracy']) + 1)
            plt.plot(epochs_range, history.history['val_accuracy'], label=label)
            plot_successful = True
        else:
            print(f"Aviso: Histórico ou 'val_accuracy' não encontrado/vazio para '{label}'.")

    if not plot_successful:
        print("Nenhum dado de acurácia de validação para plotar na comparação.")
        plt.close()
        return

    plt.title(title)
    plt.xlabel('Épocas')
    plt.ylabel('Acurácia de Validação')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True)
    plt.tight_layout(rect=[0, 0, 0.85, 1])
    plt.show()

def compare_validation_loss(histories_dict, title="Comparação de Perda de Validação (Última Execução)"):
    plt.figure(figsize=(10, 6))
    plot_successful = False
    for label, history in histories_dict.items():
        if history is not None and 'val_loss' in history.history and history.history['val_loss']:
            epochs_range = range(1, len(history.history['val_loss']) + 1)
            plt.plot(epochs_range, history.history['val_loss'], label=label)
            plot_successful = True
        else:
            print(f"Aviso: Histórico ou 'val_loss' não encontrado/vazio para '{label}'.")

    if not plot_successful:
        print("Nenhum dado de perda de validação para plotar na comparação.")
        plt.close()
        return

    plt.title(title)
    plt.xlabel('Épocas')
    plt.ylabel('Perda de Validação')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True)
    plt.tight_layout(rect=[0, 0, 0.85, 1])
    plt.show()

## 5. Definições dos Modelos

In [6]:
# Arquitetura CNN Baseline (Base)
def create_model_baseline():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(10)
    ], name="Baseline")
    return model

# CNN com Regularização L1
def create_model_l1():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(L2_FACTOR), input_shape=(32, 32, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(L2_FACTOR)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(L2_FACTOR)),
        layers.Flatten(),
        layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l1(L2_FACTOR)),
        layers.Dense(10)
    ], name="L1_Regularization")
    return model

# CNN com Regularização L2
def create_model_l2():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(L2_FACTOR), input_shape=(32, 32, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(L2_FACTOR)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(L2_FACTOR)),
        layers.Flatten(),
        layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(L2_FACTOR)),
        layers.Dense(10)
    ], name="L2_Regularization")
    return model

# CNN com Regularização L1+L2 (Elastic Net)
def create_model_l1_l2():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1_l2(l1=L2_FACTOR, l2=L2_FACTOR), input_shape=(32, 32, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1_l2(l1=L2_FACTOR, l2=L2_FACTOR)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1_l2(l1=L2_FACTOR, l2=L2_FACTOR)),
        layers.Flatten(),
        layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=L2_FACTOR, l2=L2_FACTOR)),
        layers.Dense(10)
    ], name="L1_L2_Regularization")
    return model

# CNN com Dropout
def create_model_dropout():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dropout(DROPOUT_RATE),
        layers.Dense(10)
    ], name="Dropout")
    return model

# CNN com Batch Normalization
def create_model_batch_norm():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), padding='same', use_bias=False, input_shape=(32, 32, 3)),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Flatten(),
        layers.Dense(64, use_bias=False),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Dense(10)
    ], name="Batch_Normalization")
    return model

## 6. Configuração do Data Augmentation

In [7]:
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.1
)
datagen.fit(x_train)

## 7. Loop de Treinamento para Múltiplas Execuções

In [ ]:
model_configs = {
    "Baseline": create_model_baseline,
    "L1": create_model_l1,
    "L2": create_model_l2,
    "L1_L2": create_model_l1_l2,
    "Dropout": create_model_dropout,
    "EarlyStopping": create_model_baseline,
    "BatchNorm": create_model_batch_norm,
    "DataAugmentation": create_model_baseline
}

results = {name: [] for name in model_configs.keys()}
histories_last_run = {name: None for name in model_configs.keys()}

for run in range(NUM_RUNS):
    print(f"\n--- Iniciando Execução {run + 1}/{NUM_RUNS} ---")
    seed = run
    tf.random.set_seed(seed)
    np.random.seed(seed)

    for name, model_creator in model_configs.items():
        print(f"--- Treinando modelo: {name} (Run {run + 1}) ---")
        model = model_creator()

        optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)

        model.compile(optimizer=optimizer,
                      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                      metrics=['accuracy'])

        run_callbacks = []
        if name == "EarlyStopping":
            early_stopping_callback = callbacks.EarlyStopping(
                monitor='val_loss',
                patience=PATIENCE,
                verbose=1,
                restore_best_weights=True
            )
            run_callbacks.append(early_stopping_callback)

        if name == "DataAugmentation":
            history = model.fit(datagen.flow(x_train, y_train, batch_size=BATCH_SIZE),
                                steps_per_epoch=len(x_train) // BATCH_SIZE,
                                epochs=EPOCHS,
                                validation_data=(x_val, y_val),
                                verbose=1,
                                callbacks=run_callbacks)
        else:
            history = model.fit(x_train, y_train,
                                epochs=EPOCHS,
                                batch_size=BATCH_SIZE,
                                validation_data=(x_val, y_val),
                                verbose=1,
                                callbacks=run_callbacks)

        val_loss = history.history.get('val_loss', [])
        val_acc = history.history.get('val_accuracy', [])

        if not val_acc or not val_loss:
            print(f"Aviso: Sem dados de validação (acc/loss) para {name} na execução {run + 1}. Pulando registro de resultados.")
            best_val_acc = 0
            best_val_loss = float('inf')
            best_val_acc_epoch_index = -1
            final_epoch = len(history.epoch)
        else:
            best_val_acc_epoch_index = np.argmax(val_acc)
            best_val_acc = val_acc[best_val_acc_epoch_index]
            best_val_loss = val_loss[best_val_acc_epoch_index]
            final_epoch = len(history.epoch)

        results[name].append({
            'run': run + 1,
            'best_val_acc': best_val_acc,
            'val_loss_at_best_acc': best_val_loss,
            'best_epoch': best_val_acc_epoch_index + 1 if best_val_acc_epoch_index != -1 else 0,
            'total_epochs_ran': final_epoch
        })

        if run == NUM_RUNS - 1:
            histories_last_run[name] = history
            if val_acc and val_loss:
                plot_title = f"{name} (Execução Final: {run + 1})"
                plot_history(history, plot_title)
                plot_overfitting_gap(history, plot_title)
            else:
                print(f"Não foi possível plotar o histórico individual para {name} na execução final (dados ausentes).")

## 8. Análise Quantitativa dos Resultados (Conjunto de Validação)

In [ ]:
print("\n--- Resultados Quantitativos (Conjunto de Validação) ---")
summary_data = []
for name, run_results in results.items():
    if not run_results:
        print(f"Sem resultados registrados para {name}")
        summary_data.append({
            "Técnica": name,
            f"Val Acc Média (±std)": "N/A",
            f"Melhor Época Média (±std)": "N/A",
            f"Épocas Totais Média (±std)": "N/A"
        })
        continue

    best_val_accs = [r['best_val_acc'] for r in run_results]
    best_epochs = [r['best_epoch'] for r in run_results if r['best_epoch'] > 0]
    total_epochs = [r['total_epochs_ran'] for r in run_results]

    mean_best_val_acc = np.mean(best_val_accs)
    std_best_val_acc = np.std(best_val_accs)

    mean_best_epoch = np.mean(best_epochs) if best_epochs else 0
    std_best_epoch = np.std(best_epochs) if best_epochs else 0

    mean_total_epochs = np.mean(total_epochs)
    std_total_epochs = np.std(total_epochs)

    summary_data.append({
        "Técnica": name,
        f"Val Acc Média (±std)": f"{mean_best_val_acc:.4f} ± {std_best_val_acc:.4f}",
        f"Melhor Época Média (±std)": f"{mean_best_epoch:.1f} ± {std_best_epoch:.1f}" if best_epochs else "N/A",
        f"Épocas Totais Média (±std)": f"{mean_total_epochs:.1f} ± {std_total_epochs:.1f}"
    })

summary_df = pd.DataFrame(summary_data)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
print(summary_df.to_string(index=False))

## 9. Comparação Visual (Apenas da Última Execução)

In [ ]:
print("\n--- Comparação Visual (Última Execução) ---")
if any(hist is not None for hist in histories_last_run.values()):
    compare_histories(histories_last_run)
    compare_validation_loss(histories_last_run)
else:
    print("Nenhum histórico da última execução disponível para comparação visual.")

## 10. Avaliação Final no Conjunto de Teste

In [ ]:
print(f"\n--- Avaliação Final no Conjunto de Teste ---")

acc_column_name = "Val Acc Média (±std)"

def extract_mean_acc(acc_str):
    if isinstance(acc_str, str) and "±" in acc_str:
        return float(acc_str.split(' ± ')[0])
    return -1.0

summary_df['MeanAcc'] = summary_df[acc_column_name].apply(extract_mean_acc)

if summary_df['MeanAcc'].max() == -1.0:
    print("ERRO: Não foi possível determinar o melhor modelo. Nenhuma acurácia média válida encontrada.")
    best_model_name = "Nenhum (Erro na Análise)"
    can_evaluate_best_model = False
else:
    best_model_name = summary_df.loc[summary_df['MeanAcc'].idxmax()]["Técnica"]
    print(f"Melhor modelo com base na acurácia média de validação: {best_model_name}")
    can_evaluate_best_model = True

### 10.1 Avaliação do Modelo Baseline no Teste

In [ ]:
print("\nRe-treinando e avaliando Baseline no Conjunto de Teste...")
final_seed_base = NUM_RUNS * 10
tf.random.set_seed(final_seed_base)
np.random.seed(final_seed_base)

model_baseline_final = create_model_baseline()

optimizer_base_final = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)

model_baseline_final.compile(optimizer=optimizer_base_final,
                             loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                             metrics=['accuracy'])

baseline_epochs_avg = EPOCHS
if 'Baseline' in results and results['Baseline']:
    valid_epochs = [r['best_epoch'] for r in results['Baseline'] if r['best_epoch'] > 0]
    if valid_epochs:
        baseline_epochs_avg = int(round(np.mean(valid_epochs)))
    else:
        print("Aviso: Baseline não teve 'melhor época' válida nas runs. Usando default de épocas.")

baseline_epochs_avg = max(1, baseline_epochs_avg)

print(f"Treinando baseline por {baseline_epochs_avg} épocas no conjunto completo de treino+validação...")

history_baseline_final = model_baseline_final.fit(x_train_full, y_train_full,
                                                epochs=baseline_epochs_avg,
                                                batch_size=BATCH_SIZE,
                                                verbose=0)

print("Avaliando Baseline no conjunto de teste...")
test_loss_base, test_acc_base = model_baseline_final.evaluate(x_test, y_test, verbose=2)
print(f"-> Acurácia Final do Baseline no Teste: {test_acc_base:.4f}")

### 10.2 Avaliação do Melhor Modelo no Teste

In [ ]:
if can_evaluate_best_model and best_model_name in model_configs:
    print(f"\nRe-treinando e avaliando Melhor Modelo ({best_model_name}) no Conjunto de Teste...")
    final_seed_best = NUM_RUNS * 10 + 1
    tf.random.set_seed(final_seed_best)
    np.random.seed(final_seed_best)

    best_model_creator = model_configs[best_model_name]
    model_best_final = best_model_creator()

    optimizer_best_final = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)

    model_best_final.compile(optimizer=optimizer_best_final,
                            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                            metrics=['accuracy'])

    best_model_epochs_avg = EPOCHS
    if best_model_name in results and results[best_model_name]:
        valid_epochs_best = [r['best_epoch'] for r in results[best_model_name] if r['best_epoch'] > 0]
        if valid_epochs_best:
            best_model_epochs_avg = int(round(np.mean(valid_epochs_best)))
        else:
            print(f"Aviso: {best_model_name} não teve 'melhor época' válida nas runs. Usando default de épocas.")

    best_model_epochs_avg = max(1, best_model_epochs_avg)

    print(f"Treinando {best_model_name} por {best_model_epochs_avg} épocas no conjunto completo de treino+validação...")

    final_callbacks = []

    if best_model_name == "DataAugmentation":
        datagen_final = ImageDataGenerator(
            rotation_range=15,
            width_shift_range=0.1,
            height_shift_range=0.1,
            horizontal_flip=True,
            zoom_range=0.1
        )
        print(f"Usando Data Augmentation para o treino final de {best_model_name}.")
        history_best_final = model_best_final.fit(datagen_final.flow(x_train_full, y_train_full, batch_size=BATCH_SIZE),
                                                    steps_per_epoch=len(x_train_full) // BATCH_SIZE,
                                                    epochs=best_model_epochs_avg,
                                                    verbose=0,
                                                    callbacks=final_callbacks)
    else:
        history_best_final = model_best_final.fit(x_train_full, y_train_full,
                                                    epochs=best_model_epochs_avg,
                                                    batch_size=BATCH_SIZE,
                                                    verbose=0,
                                                    callbacks=final_callbacks)

    print(f"Avaliando {best_model_name} no conjunto de teste...")
    test_loss_best, test_acc_best = model_best_final.evaluate(x_test, y_test, verbose=2)
    print(f"-> Acurácia Final do {best_model_name} no Teste: {test_acc_best:.4f}")

elif not can_evaluate_best_model:
    print("\nAvaliação do melhor modelo pulada devido a erro na determinação do melhor modelo.")
else:
    print(f"\nAvaliação do melhor modelo ({best_model_name}) pulada (modelo não encontrado nas configurações?).")